# Translate all verbs

In [ ]:
gpt_eng_translations = []
for verb in darija_verbs:
    completion = client.chat.completions.create(
    model =  "gpt-3.5-turbo",
    messages = [
    {
        "role": "system",
        "content": "You will be provided with a verb in Moroccan Darija language," +
        " and your task is to translate it into English. Do your best with one output word only, unless it's a phrasal verb."
    },
    {
        "role": "user", 
        "content":  f'{verb}' 
    }
    ],
    temperature=0.3, # deterministic
   # max_tokens=1, # in case of a dot
    )
    gpt_eng_translations.append(completion.choices[0].message.content)

print(gpt_eng_translations)

# metrics for word similarity:

- Word Similarity Metrics: 
Methods such as cosine similarity between word embeddings (vectors representing words) from models like Word2Vec, GloVe, or BERT can be used. These models capture semantic properties of words, so similar words have similar vectors.


- WordNet-Based Similarity: 
WordNet is a lexical database for the English language that groups words into sets of synonyms (synsets) and captures various semantic relations between these sets. Libraries like NLTK in Python provide access to WordNet and functions to compute semantic similarity scores between words based on their synsets' relationships within the WordNet hierarchy. Metrics like path similarity, Wu-Palmer similarity, and Leacock-Chodorow similarity are examples of WordNet-based measures.

- Edit Distance: 
For a more syntactic comparison, edit distance (Levenshtein distance) measures how many single-character edits (insertions, deletions, substitutions) are required to change one word into another. While not a measure of semantic similarity, it's useful for evaluating spelling or typing errors and morphological similarity.


###  >> chosen here: wornet synsets, cosine?

# metrics for sentence similarity (semantic comparison)

- BLEU Score (Bilingual Evaluation Understudy): 
A standard metric for evaluating a generated text to reference texts. BLEU measures the similarity of the machine translations to one or more reference translations, considering precision of n-grams (sequences of n words).

- ROUGE Score (Recall-Oriented Understudy for Gisting Evaluation): 
Especially useful for evaluating summarization tasks, ROUGE measures overlap between the generated text and the reference texts, including n-gram overlap, word sequence overlap, and word pair overlap.

- METEOR Score (Metric for Evaluation of Translation with Explicit ORdering): 
Attempts to improve upon BLEU by considering synonyms, stemming, and paraphrase matching, as well as sequence alignment for translation evaluation. It balances precision and recall, and it's designed to address some of BLEU's shortcomings.

- TER (Translation Edit Rate): 
Measures the number of edits required to change a system output into one of the references. TER is an error metric where lower scores are better, indicating fewer modifications needed for the translation to match a reference.

- BERTScore: 
Leverages the pre-trained contextual embeddings from BERT (or other transformers) and matches words in candidate and reference sentences by cosine similarity. It's robust against paraphrases since it considers the context of words.

- ChrF (Character n-gram F-score): 
Focuses on character n-grams rather than word n-grams, which can be particularly useful for languages with rich morphology or when dealing with agglutinative languages. It's also useful for languages with less training data available.

### >> Metrics chosen: BLEU, METEOR, TER, BERTScore, ChrF (few data)

## Synsets concept
why synsets and not synonyms?
- Synsets: These are groups of synonyms that share a common meaning. Each synset represents a unique concept and includes a set of synonyms (lemmas) that are interchangeable in some context. Synsets are interconnected by various semantic relations, including hypernyms (more general concepts), hyponyms (more specific concepts), antonyms (opposites), and more.

In [21]:
from nltk.corpus import wordnet as wn

synsets1 = wn.synsets('go_out', pos=wn.VERB)
for synset in synsets1:
    print(synset)


Synset('exit.v.01')
Synset('go_out.v.02')
Synset('go_out.v.03')
Synset('go_out.v.04')
Synset('go_out.v.05')
Synset('go_steady.v.01')


- exit, go_out: lemmas of the synset
- v: verb, n: noun, a: adjective, s: satellite adj, r: adverb.
- 01/02..: a seperate meaning of that lemma.

## path similarity
How it work in WordNet?

The path_similarity function returns a score between 0 and 1, where 1 means the synsets are identical, and a lower score indicates less similarity.

The similarity is calculated as follows:

similarity = 1 / (path length + 1) ​
 
Path length : is the number of edges between the two synsets in the WordNet graph. 

### examples of path similarity between words to determine threshold:

In [32]:
from nltk.corpus import wordnet as wn

def max_path_similarity(word1, word2):
    """Calculate the maximum path similarity between two words."""
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)
    max_score = 0
    for syn1 in synsets1:
        for syn2 in synsets2:
            score = syn1.path_similarity(syn2)
            if score and score > max_score:
                max_score = score
    return max_score

# List of word pairs categorized by their expected similarity
word_pairs = [
    # Very high similarity - almost synonyms
    ("coast", "shore"),
    ("journey", "voyage"),
    ("car", "automobile"),
    
    # Moderate to low similarity - part-whole relationship or closely similar concepts
    ("python", "java"),
    ("forest", "tree"),
    
    # Antonyms or distinct but related concepts
    ("love", "hate"),
    ("boy", "girl"),
]

# Calculate and print the path similarity for each pair
for word1, word2 in word_pairs:
    similarity = max_path_similarity(word1, word2)
    print(f"The maximum path similarity between '{word1}' and '{word2}' is: {similarity:.4f}")


The maximum path similarity between 'coast' and 'shore' is: 0.5000
The maximum path similarity between 'journey' and 'voyage' is: 0.5000
The maximum path similarity between 'car' and 'automobile' is: 1.0000
The maximum path similarity between 'python' and 'java' is: 0.0714
The maximum path similarity between 'forest' and 'tree' is: 0.3333
The maximum path similarity between 'love' and 'hate' is: 0.3333
The maximum path similarity between 'boy' and 'girl' is: 0.2000


#### Similarity looks right subjectively =>0.5
#### >> instances from the verbs csv file

In [33]:
from nltk.corpus import wordnet as wn
from nltk import download

# Download necessary NLTK data
download('wordnet')

def word_similarity(word1, word2):
    """Computing the maximum path similarity score between two words."""
    synsets1 = wn.synsets(word1,)
    synsets2 = wn.synsets(word2)
    
    max_score = 0
    for syn1 in synsets1:
        for syn2 in synsets2:
            score = syn1.path_similarity(syn2)
            if score and score > max_score:
                max_score = score
    return max_score

def evaluate_pairs(word_pairs, threshold=0.5):
    """Evaluating each pair using WordNet path similarity and a threshold."""
    results = []
    for word1, word2 in word_pairs:
        similarity = word_similarity(word1, word2)
        results.append((word1, word2, similarity, similarity >= threshold))
    return results

# Example word pairs
word_pairs = [
    ("go out", "exit"),
    ("understand", "comprehend"),
    ("explain", "clarify"),
    ("learn", "study"),
    ("teach", "instruct"),
    ("hear", "listen"),
    ("see", "observe"),
    ("smell", "scent"),
    ("taste", "flavor"),
    ("touch", "feel"),
    ("go", "move"),
    ("come", "arrive"),
    ("ascend", "rise"),
    ("go down", "descend")
]

# Evaluate the pairs
evaluated_pairs = evaluate_pairs(word_pairs)

# Print the results
for word1, word2, similarity, is_similar in evaluated_pairs:
    print(f"{word1} - {word2}: Similarity = {similarity:.2f}, Similar = {is_similar}")


go out - exit: Similarity = 0.00, Similar = False
understand - comprehend: Similarity = 0.50, Similar = True
explain - clarify: Similarity = 0.50, Similar = True
learn - study: Similarity = 1.00, Similar = True
teach - instruct: Similarity = 1.00, Similar = True
hear - listen: Similarity = 1.00, Similar = True
see - observe: Similarity = 0.50, Similar = True
smell - scent: Similarity = 1.00, Similar = True
taste - flavor: Similarity = 0.50, Similar = True
touch - feel: Similarity = 0.50, Similar = True
go - move: Similarity = 1.00, Similar = True
come - arrive: Similarity = 1.00, Similar = True
ascend - rise: Similarity = 1.00, Similar = True
go down - descend: Similarity = 0.00, Similar = False


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yassi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### >> phrasal verbs need further cleaning  from "go out" to "go_out"

# Next steps:
Can work on a different ipynb if this is not showing.
- phrasal verbs in results.cv turned into verbs with tirets 8: "_"
- cleaning to .. verbs. >> must redo the instruction for openai to more clear not including to + verb.
- applying the path similarity function on the synsets.
- checks on True percentage of correct match and draw it in a py chart with plotly.
- compare another csv containing sentences.
- evaluate with cosine similarity, words? if I have time or if I asked professor.

In [1]:
from nltk.corpus import wordnet as wn

synsets1 = wn.synsets('to_go', pos=wn.VERB)
for synset in synsets1:
    print(synset)